In [ ]:
import pandas as pd

# ^^^ pyforest auto-imports - don't write above this line
sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/")
sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/bib_lookup/")

import json
import pickle
from collections import Counter
from copy import deepcopy

import wfdb
from tqdm.auto import tqdm

from torch_ecg.utils import list_sum
from scipy.io import loadmat
from torch_ecg.components.outputs import (
    ClassificationOutput,
)
from torch_ecg.utils.misc import dict_to_str
from torch_ecg.cfg import CFG, DEFAULTS

from utils.scoring_metrics import compute_challenge_metrics, compute_challenge_score
from utils.features import get_features, get_labels
from utils.misc import load_challenge_metadata
from cfg import TrainCfg, ModelCfg, BaseCfg
from data_reader import CINC2023Reader
from dataset import CinC2023Dataset
from outputs import CINC2023Outputs, cpc2outcome_map
from models import CRNN_CINC2023
from trainer import CINC2023Trainer, _set_task
from helper_code import (
    find_data_folders,
    load_challenge_data,
)

%load_ext autoreload
%autoreload 2

## DL model

In [ ]:
CINC2023Trainer.__DEBUG__ = False
CRNN_CINC2023.__DEBUG__ = False
CinC2023Dataset.__DEBUG__ = False

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if ModelCfg.torch_dtype == torch.float64:
    torch.set_default_tensor_type(torch.DoubleTensor)
    DTYPE = np.float64
else:
    DTYPE = np.float32

In [ ]:
TASK = "classification"

train_config = deepcopy(TrainCfg)
# train_config.db_dir = data_folder
# train_config.model_dir = model_folder
# train_config.final_model_filename = _ModelFilename
train_config.debug = True

train_config.db_dir = "/home/wenh06/Jupyter/wenhao/data/CinC2023/training/"
# train_config.db_dir = "/home/wenh06/Jupyter/wenhao/data/CinC2023/training_subset/"

# train_config.n_epochs = 100
# train_config.batch_size = 24  # 16G (Tesla T4)
# train_config.log_step = 20
# # train_config.max_lr = 1.5e-3
# train_config.early_stopping.patience = 20

train_config[TASK].model_name = "crnn"  # "crnn"

train_config[
    TASK
].cnn_name = "resnet_nature_comm_bottle_neck_se"  # "resnet_nature_comm_bottle_neck_se"
# train_config[TASK].rnn_name = "none"  # "none", "lstm"
# train_config[TASK].attn_name = "se"  # "none", "se", "gc", "nl"

_set_task(TASK, train_config)

model_config = deepcopy(ModelCfg[TASK])

# adjust model choices if needed
model_config.model_name = train_config[TASK].model_name
# print(model_name)
if "cnn" in model_config[model_config.model_name]:
    model_config[model_config.model_name].cnn.name = train_config[TASK].cnn_name
if "rnn" in model_config[model_config.model_name]:
    model_config[model_config.model_name].rnn.name = train_config[TASK].rnn_name
if "attn" in model_config[model_config.model_name]:
    model_config[model_config.model_name].attn.name = train_config[TASK].attn_name

In [ ]:
model = CRNN_CINC2023(config=model_config)
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model = model.to(device=DEVICE)

In [ ]:
model.module_size, model.module_size_

In [ ]:
model

In [ ]:
ds_train = CinC2023Dataset(train_config, TASK, training=True, lazy=True)
ds_test = CinC2023Dataset(train_config, TASK, training=False, lazy=True)

In [ ]:
ds_test._load_all_data()

In [ ]:
ds_train._load_all_data()

In [ ]:
ds_test.cache["waveforms"].shape

In [ ]:
ds_train.cache["cpc"]

In [ ]:
trainer = CINC2023Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=DEVICE,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_test)

In [ ]:
best_state_dict = trainer.train()

## ML model

In [ ]:
verbose = 2
data_folder = "/home/wenh06/Jupyter/wenhao/data/CinC2023/training/"

In [ ]:
patient_ids = find_data_folders(data_folder)
num_patients = len(patient_ids)

if num_patients == 0:
    raise FileNotFoundError("No data was provided.")
else:
    if verbose >= 1:
        print(f"Found {num_patients} patients.")

In [ ]:
if verbose >= 1:
    print("Extracting features and labels from the Challenge data...")

features = list()
outcomes = list()
cpcs = list()

for i in tqdm(
    range(num_patients),
    desc="Extracting features and labels",
    total=num_patients,
    dynamic_ncols=True,
    mininterval=1.0,
    disable=verbose < 2,
):
    # Load data.
    patient_id = patient_ids[i]
    patient_metadata = load_challenge_metadata(data_folder, patient_id)

    # Extract features.
    current_features = get_features(patient_metadata)
    features.append(current_features)

    # Extract labels.
    current_labels = get_labels(patient_metadata)
    outcomes.append(current_labels["outcome"])
    cpcs.append(current_labels["cpc"])

features = np.vstack(features)
outcomes = np.vstack(outcomes)
cpcs = np.vstack(cpcs)

In [ ]:
# Define parameters for random forest classifier and regressor.
n_estimators = 42  # Number of trees in the forest.
max_leaf_nodes = 456  # Maximum number of leaf nodes in each tree.
random_state = 789  # Random state; set for reproducibility.

# Impute any missing features; use the mean value by default.
imputer = SimpleImputer().fit(features)

# Train the models.
features = imputer.transform(features)
outcome_model = RandomForestClassifier(
    n_estimators=n_estimators,
    max_leaf_nodes=max_leaf_nodes,
    random_state=random_state,
).fit(features, outcomes.ravel())
cpc_model = RandomForestRegressor(
    n_estimators=n_estimators,
    max_leaf_nodes=max_leaf_nodes,
    random_state=random_state,
).fit(features, cpcs.ravel())

d = {"imputer": imputer, "outcome_model": outcome_model, "cpc_model": cpc_model}

In [ ]:
# with open("./tmp/final_model_ml.pkl", "wb") as f:
#     pickle.dump(d, f)